In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pesurrcampusmicompetitionb/sample_submission.csv
/kaggle/input/pesurrcampusmicompetitionb/train.csv
/kaggle/input/pesurrcampusmicompetitionb/test.csv


In [2]:
import os
print(os.getcwd())

/kaggle/working


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [4]:
# Load the datasets
train_data = pd.read_csv('/kaggle/input/pesurrcampusmicompetitionb/train.csv')
test_data = pd.read_csv('/kaggle/input/pesurrcampusmicompetitionb/test.csv')

# Assuming 'label' is the target column
X = train_data.drop(['filename', 'label'], axis=1)
y = train_data['label']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Deep Learning model (Artificial Neural Network)
input_dim = X_train.shape[1]
output_dim = len(train_data['label'].unique())

y_train_categorical = keras.utils.to_categorical(y_train)  # One-hot encode the labels
y_val_categorical = keras.utils.to_categorical(y_val)

# Neural Network Architecture
ann_model = Sequential()
ann_model.add(Dense(512, input_dim=input_dim, activation='relu'))
ann_model.add(Dropout(0.5))  # Dropout for regularization
ann_model.add(Dense(256, activation='relu'))
ann_model.add(Dropout(0.5))  # Dropout for regularization
ann_model.add(Dense(output_dim, activation='softmax'))

# Model Compilation
ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
history = ann_model.fit(X_train, y_train_categorical, epochs=120, batch_size=128, validation_data=(X_val, y_val_categorical))

# Evaluate the model on validation data
_, accuracy = ann_model.evaluate(X_val, y_val_categorical)

# Preprocess test data
test_features = test_data.drop('id', axis=1)
test_features_scaled = scaler.transform(test_features)

# Predict probabilities on test set
test_probabilities = ann_model.predict(test_features_scaled)

# Extract predicted labels based on highest probability
test_predictions = test_probabilities.argmax(axis=1)

# Create a DataFrame with 'id' and 'label' columns for submission
submission_df = pd.DataFrame({'id': test_data['id'], 'label': test_predictions})

# Save predictions as a CSV file
submission_df.to_csv('submission.csv', index=False)

Epoch 1/120
57/57 [==============================] - 2s 12ms/step - loss: 1.5677 - accuracy: 0.4378 - val_loss: 1.1351 - val_accuracy: 0.6076
Epoch 2/120
57/57 [==============================] - 0s 8ms/step - loss: 1.1590 - accuracy: 0.5843 - val_loss: 0.9505 - val_accuracy: 0.6793
Epoch 3/120
57/57 [==============================] - 0s 9ms/step - loss: 0.9883 - accuracy: 0.6507 - val_loss: 0.8306 - val_accuracy: 0.7315
Epoch 4/120
57/57 [==============================] - 0s 8ms/step - loss: 0.9098 - accuracy: 0.6796 - val_loss: 0.7704 - val_accuracy: 0.7460
Epoch 5/120
57/57 [==============================] - 0s 8ms/step - loss: 0.8441 - accuracy: 0.7037 - val_loss: 0.7254 - val_accuracy: 0.7643
Epoch 6/120
57/57 [==============================] - 0s 9ms/step - loss: 0.7821 - accuracy: 0.7297 - val_loss: 0.6777 - val_accuracy: 0.7854
Epoch 7/120
57/57 [==============================] - 0s 8ms/step - loss: 0.7651 - accuracy: 0.7404 - val_loss: 0.6550 - val_accuracy: 0.8010
Epoch 8/120
